In [80]:
from bokeh.plotting import figure
from bokeh.io import output_file, show, output_notebook
from bokeh.models import ColumnDataSource
from bokeh.models.tools import HoverTool
import pandas

In [81]:
data_orange_pi_st = pandas.read_csv("sbc_performance_data/orange_pi/zero_lts/buster/singlethread_output.csv")
data_raspberry_pi_st = pandas.read_csv("sbc_performance_data/raspberry_pi/buster/4B/std_kernel_4B_singlethread_output.csv")
data_rockpi_st = pandas.read_csv("sbc_performance_data/rockpi_s/stretch_4.4.143/singlethread_output.csv")

In [82]:
data_raspberry_pi_st.dtypes

time              int64
queen             int64
threads           int64
solutions         int64
seconds           int64
microseconds      int64
cpu_temp        float64
gpu_temp        float64
dtype: object

In [83]:
data_raspberry_pi_st.head()


,time,queen,threads,solutions,seconds,microseconds,cpu_temp,gpu_temp
0,1566160128,12,1,14200,66,943882,61.84,61.0
1,1566160197,12,1,14200,68,964935,63.78,63.0
2,1566160263,12,1,14200,66,262455,64.27,64.0
3,1566160330,12,1,14200,66,700631,64.76,64.0
4,1566160396,12,1,14200,66,150964,64.76,64.0


In [84]:
x = data_raspberry_pi_st['time']
y = data_raspberry_pi_st['microseconds']

o_x = data_orange_pi_st['time']
o_y = data_orange_pi_st['microseconds']

r_x = data_rockpi_st['time']
r_y = data_rockpi_st['microseconds']

In [89]:
xmin = x.min()
oxmin = o_x.min()
rxmin = r_x.min()

x_o_x_time = x.apply(lambda x: x - xmin).combine_first(o_x.apply(lambda x: x - oxmin)).combine_first(r_x.apply(lambda x: x - rxmin))
x_o_x_time

0         0.0
1        69.0
2       135.0
3       202.0
4       268.0
5       335.0
6       402.0
7       467.0
8       535.0
9       601.0
10      670.0
11      738.0
12      805.0
13      871.0
14      937.0
15     1003.0
16     1071.0
17     1137.0
18     1205.0
19     1273.0
20     1340.0
21     1408.0
22     1474.0
23     1541.0
24     1608.0
25     1674.0
26     1741.0
27     1806.0
28     1874.0
29     1940.0
30     2007.0
31     2073.0
32     2140.0
33     2207.0
34     2275.0
35     2341.0
36     2409.0
37     2475.0
38     2541.0
39     2607.0
40     2674.0
41     2740.0
42     2806.0
43     2872.0
44     2939.0
45    15138.0
46    15478.0
47    15815.0
48    16151.0
49    16486.0
Name: time, dtype: float64

In [97]:
output_notebook()

source = ColumnDataSource(data=dict(
    y=x_o_x_time,
    x1=y,
    x2=o_y,
    x3=r_y,
))

p = figure()
p.add_tools(HoverTool())
p.background_fill_color = "grey"
p.hbar_stack(['x1', 'x2', 'x3'], y='y', height=0.8, color=("red", "orange", "yellow"), source=source)
#p.scatter(x = 'x1', y='y',size=20, source=source)
show(p)


Loading BokehJS ...